# 生成されたサンプルを辞書に基づいて日本語に直す

seqGANで生成されたサンプルを辞書に基づいて日本語の文字列に変換する

また、変換する際には、seqGANの識別機が算出したサンプルに対する本物かどうかの確率と紐付ける

これは、生成されたキャッチコピーに対する自然さの度合いを表していると考えられ、キャッチコピーに対するスコア付けの手法の一つとして用いる

今回は博多駅を対象にしており、他の観光地にて生成を行う場合は、適当に合わせること

In [1]:
# 生成されたサンプルを辞書に基づいて日本語に戻す
import pickle

# seqGANにて生成した結果を読み込む
with open('data/generated/hakataeki/output/output_text_200.txt') as f:
    s = f.readlines()

# seqGANの生成結果に対する識別器の確率を読み込む
with open('data/generated/hakataeki/output/prob_200.txt') as f:
    prob = f.readlines()

# 生成結果はid列なので日本語に戻すために作成した辞書を読み込む
with open('data/dict/all_dict.pickle', 'rb') as f:
    indices_char = pickle.load(f)

temp_list = []

for i, j in zip(s, prob):
    temp = [i,j]
    temp_list.append(temp)

print("生成数",len(temp_list))

num_prob_list = []

for k in temp_list:
    if k not in num_prob_list:
        num_prob_list.append(k)

print("重複をなくした生成数",len(num_prob_list))




# # 順番を保持したまま重複をなくす
# s = sorted(set(s), key=s.index)
# print(len(s))

haiku_prob_list = []
haiku_temp = []
result = ""
for j in range(len(num_prob_list)):
    line = num_prob_list[j][0].split()
    line = [int(i) for i in line]
    # print(line)
    # 単語に対応するidを並べて行く
    for i in line:
        if i > len(indices_char):
            continue
        # print(char_indices[i])
        result += str(indices_char[i])
        result += " "
    # result += "0" # 文末記号を追加
    result = result.replace('_', '')
    result = result.replace(' ', '')
    # result += "\n"
    haiku_temp.append(result)
    result = ""
# print(haiku_temp[:10])
for p in range(len(haiku_temp)):
    num_prob_list[p][1] = num_prob_list[p][1][:-1]
    temp = [haiku_temp[p],num_prob_list[p][1]]
    haiku_prob_list.append(temp)

print(len(haiku_prob_list))
print("\n")
print(haiku_prob_list[:10])


生成数 6400
重複をなくした生成数 464
464


[['アクセスが良い駅だなと思いまし', '0.9576534'], ['駅前の道路の工事渋滞で', '0.8326542'], ['お土産は１Fみやげもん市場', '0.70703167'], ['イベントもあるし１日遊べます', '0.66840965'], ['良いのが難点ですがアクセスも開催て', '0.6257231'], ['雰囲気が良い駅だなと思いまし', '0.55826056'], ['粥が良い駅だなと思いまし', '0.54197824'], ['シーズンになるとたくさんありました', '0.25650907'], ['地下鉄も新幹線でつながって', '0.21183991'], ['全体が数年前にリニューアルられ１日思います', '0.19926642']]


日本語に変換したサンプルが575の音韻を満たしているかフィルタリングを行う

やってること自体は、レビューから575の文字列を抽出しているのと同じ

In [2]:
# with open('generated/sepa/hakataeki/content/save/output/output_2_haiku.txt') as f:
#     s = f.readlines()

import MeCab
tagger = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd") #mecab neologd


# text_list = []

# for line in s:
#     line = line.replace(" ", "")
#     text_list.append(line)

# print(text_list)




haiku_list = []
for x,y in haiku_prob_list:
    temp = x
    # print(temp)

    word_list = []

    # 形態素解析をして、単語毎にリストに格納
    for word in tagger.parse(temp).splitlines():
        word = word.replace('\t', ',')
        # print(word)
        temp_word_list = word.split(',')
        word_list.append(temp_word_list)

    # 文末の記号を削除
    word_list.pop(-1)

    # print(word_list)

    # 格納された単語を繋げて17音の文字列を作る
    for j in range(len(word_list)):

        # 最初に空の文字列を用意
        temp_haiku = ""
        part = ""
        # 次の単語を示すカウント
        count = 0
        # 音を数えるcount
        haku_count = 0

        # 17音を超えるまで単語を繋げる

        if not ('名詞' == word_list[j+count][1] or '動詞' == word_list[j+count][1] or '形容詞' == word_list[j+count][1]):
            continue
        if ('名詞' ==  word_list[j+count][1] and ('非自立' == word_list[j+count][2] or '接尾' == word_list[j+count][2])):
            continue
        if ('動詞' ==  word_list[j+count][1] and '非自立' == word_list[j+count][2]):
            continue

        # 上の句5
        while haku_count < 5:
            # 単語を繋げる
            part =part + word_list[j + count][0]
            # 音を記録

            if '記号' not in word_list[j+count][1]:
                temp = word_list[j+count][-1]

                # 小さい文字は音にならないので消去
                # temp = temp.replace('ッ', '')
                temp = temp.replace('ャ', '')
                temp = temp.replace('ュ', '')
                temp = temp.replace('ョ', '')
                temp = temp.replace('ァ', '')
                temp = temp.replace('ィ', '')
                temp = temp.replace('ゥ', '')
                temp = temp.replace('ェ', '')
                temp = temp.replace('ォ', '')

                haku_count += len(temp)
            # 元の文が終わっていたら終了
            if j + count >= len(word_list)-1:
                break
            count += 1
            # print(part, haku_count)
        
        if haku_count == 5:
            temp_haiku += part
            # temp_haiku += "\n"
            part = ""
            haku_count = 0
        else:
            continue

        if not ('名詞' == word_list[j+count][1] or '動詞' == word_list[j+count][1] or '形容詞' == word_list[j+count][1]):
            continue
        if ('名詞' ==  word_list[j+count][1] and ('非自立' == word_list[j+count][2] or '接尾' == word_list[j+count][2])):
            continue
        if ('動詞' ==  word_list[j+count][1] and '非自立' == word_list[j+count][2]):
            continue

        # 中の句7
        while haku_count < 7:
            # 単語を繋げる
            part =part + word_list[j + count][0]
            # 音を記録

            if '記号' not in word_list[j+count][1]:
                temp = word_list[j+count][-1]

                # 小さい文字は音にならないので消去
                # temp = temp.replace('ッ', '')
                temp = temp.replace('ャ', '')
                temp = temp.replace('ュ', '')
                temp = temp.replace('ョ', '')
                temp = temp.replace('ァ', '')
                temp = temp.replace('ィ', '')
                temp = temp.replace('ゥ', '')
                temp = temp.replace('ェ', '')
                temp = temp.replace('ォ', '')

                haku_count += len(temp)
            # 元の文が終わっていたら終了
            if j + count >= len(word_list)-1:
                break
            count += 1
            # print(temp_haiku, haku_count)

        
        
        if haku_count == 7:
            temp_haiku += part
            # temp_haiku += "\n"
            part = ""
            haku_count = 0
        else:
            continue

        if not ('名詞' == word_list[j+count][1] or '動詞' == word_list[j+count][1] or '形容詞' == word_list[j+count][1]):
            continue
        if ('名詞' ==  word_list[j+count][1] and ('非自立' == word_list[j+count][2] or '接尾' == word_list[j+count][2])):
            continue
        if ('動詞' ==  word_list[j+count][1] and '非自立' == word_list[j+count][2]):
            continue

        # 下の句5
        while haku_count < 5:
            # 単語を繋げる
            part =part + word_list[j + count][0]
            # 音を記録

            if '記号' not in word_list[j+count][1]:
                temp = word_list[j+count][-1]

                # 小さい文字は音にならないので消去
                # temp = temp.replace('ッ', '')
                temp = temp.replace('ャ', '')
                temp = temp.replace('ュ', '')
                temp = temp.replace('ョ', '')
                temp = temp.replace('ァ', '')
                temp = temp.replace('ィ', '')
                temp = temp.replace('ゥ', '')
                temp = temp.replace('ェ', '')
                temp = temp.replace('ォ', '')

                haku_count += len(temp)
            # 元の文が終わっていたら終了
            if j + count >= len(word_list)-1:
                break
            count += 1
            # print(temp_haiku, haku_count)

        
        if haku_count == 5:
            temp_haiku += part
            # temp_haiku += "\n"
            # temp_haiku += "\n"
            part = ""
            haku_count = 0
        else:
            continue
        

        if temp_haiku not in haiku_list:
            haiku_list.append([temp_haiku,y])

print(len(haiku_list))

print(haiku_list[:10])


136
[['アクセスが良い駅だなと思いまし', '0.9576534'], ['駅前の道路の工事渋滞で', '0.8326542'], ['お土産は１Fみやげもん市場', '0.70703167'], ['イベントもあるし１日遊べます', '0.66840965'], ['雰囲気が良い駅だなと思いまし', '0.55826056'], ['シーズンになるとたくさんありました', '0.25650907'], ['地下鉄も新幹線でつながって', '0.21183991'], ['全体が数年前にリニューアル', '0.19926642'], ['福岡の窓口である博多駅', '0.14426497'], ['福岡の美味しいものが揃ってい', '0.1317006']]


# スコア付け

フィルタリングを行ったサンプルに対して、３つの手法でスコア付けを行う

- tf-idfを用いたスコア付け　⇨　観光地の特徴度を表す
- せqGANの識別器が算出したキャッチコピーに対する本物かどうかの確率を用いる　⇨　キャッチコピーの自然さを表す
- 先の２つのスコアを組み合わせたスコア付け　⇨　キャッチコピーの全体的な完成度を表す

# seqGANの識別器を用いたスコア付け

seqGANでは生成器が生成したキャッチコピーに対して、識別器がそれが本物かどうかの確率を算出する。

この時、識別器が本物だと判断すると、生成器は識別器を騙すことができており、自然なキャッチコピーが生成できていると考えられる。

したがって、識別器の確率は、キャッチコピーの自然さを表していると考えられる。

# tf-idfを用いたスコア付け

tf-idfを用いたスコア付けでは、サンプルに含まれる名詞に対するtf-idfの値の合計をキャッチコピーのスコアとする

これはキャッチコピーが持つ観光地の特徴度合いを表していると考えられる

In [8]:
#-------------------------------------------------データベース----------------------------------------

import sqlite3

db_word_list = []

conn = sqlite3.connect('data/tfidf_db.db')

c = conn.cursor()

c.execute('select word from hakataeki')
db_words = c.fetchall()

for word in db_words:
    db_word_list.append(word[0])

# print(db_word_list)

#-------------------------------------------------tfidfのスコア付け----------------------------------------
score_list = []

for i,_ in haiku_list:
    temp = i
    # print(temp)

    word_list = []
    score_sum = 0

    # 形態素解析をして、単語毎にリストに格納
    for word in tagger.parse(temp).splitlines():
        word = word.replace('\t', ',')
        temp_word_list = word.split(',')
        word_list.append(temp_word_list)

    # 文末の記号を削除
    word_list.pop(-1)

    

    for j in range(len(word_list)):
        # print(word_list[j][0])

        temp_score = 0

        if '名詞' not in word_list[j][1]:
            continue

        # 形態素解析した名詞がdbに含まれているならtfidfの値をスコアとして加算していく
        if word_list[j][0] in db_word_list:
            c.execute('select tfidf from hakataeki where word = ?', (word_list[j][0],))
            tfidf = c.fetchone()[0]
            temp_score = tfidf
        
        score_sum += temp_score

    score_list.append(score_sum)

        

# print(score_list)

#-------------------------------------------------データベース----------------------------------------

#閉じる
conn.close()





[0.0, 0.0033141180043055746, 0.018665275928839205, 0.00022918083362860256, 0.0, 0.0, 0.019500547406398814, 0.0007971909090526924, 0.00020799975214914443, 0.0, 0.0057355019333819425, 0.008381355684012938, 0.016549528840836197, 0.0005627747554481164, 0.0007521069133698054, 0.014247983022216394, 0.019500547406398814, 0.008381355684012938, 0.008381355684012938, 0.010981352585877244, 0.0, 0.000637516496555504, 0.00029159146122394435, 0.002047021129348379, 0.0025609697923372867, 0.0030266006636713388, 0.00023281543566702607, 0.019500547406398814, 0.0020953389210032346, 0.0, 0.00469533582286754, 2.864760420357532e-05, 0.008381355684012938, 0.0020953389210032346, 0.008482523949407184, 0.0, 0.00046563087133405214, 0.005564564012406137, 0.019500547406398814, 0.00035947721037082403, 0.0, 0.014247983022216394, 0.001221569705389551, 0.0007776304995577687, 0.0025609697923372867, 0.0, 0.0025999969018643053, 0.0005627747554481164, 0.00022918083362860256, 0.00041599950429828885, 0.014247983022216394, 0

In [16]:
# 俳句とtfidfのスコアをまとめたリストを作成

for i in range(len(haiku_list)):
    haiku_list[i].append(score_list[i])

print("生成したキャッチコピー数：",len(haiku_list))
print("")
print("生成例")
for n in haiku_list[:10]:
    print("キャッチコピー:", n[0], " \t\t","識別器の確率:",n[1], "\t\t", "tf-idf値:",n[2])

# print(haiku_list)

# import pickle

# with open('data/generated/sepa/dazaifu/content/save/output/haiku_dis_tfidf.pickle', 'wb') as f:
#     pickle.dump(haiku_list, f)

生成したキャッチコピー数： 136

生成例
キャッチコピー: アクセスが良い駅だなと思いまし  		 識別器の確率: 0.9576534 		 tf-idf値: 0.0
キャッチコピー: 駅前の道路の工事渋滞で  		 識別器の確率: 0.8326542 		 tf-idf値: 0.0033141180043055746
キャッチコピー: お土産は１Fみやげもん市場  		 識別器の確率: 0.70703167 		 tf-idf値: 0.018665275928839205
キャッチコピー: イベントもあるし１日遊べます  		 識別器の確率: 0.66840965 		 tf-idf値: 0.00022918083362860256
キャッチコピー: 雰囲気が良い駅だなと思いまし  		 識別器の確率: 0.55826056 		 tf-idf値: 0.0
キャッチコピー: シーズンになるとたくさんありました  		 識別器の確率: 0.25650907 		 tf-idf値: 0.0
キャッチコピー: 地下鉄も新幹線でつながって  		 識別器の確率: 0.21183991 		 tf-idf値: 0.019500547406398814
キャッチコピー: 全体が数年前にリニューアル  		 識別器の確率: 0.19926642 		 tf-idf値: 0.0007971909090526924
キャッチコピー: 福岡の窓口である博多駅  		 識別器の確率: 0.14426497 		 tf-idf値: 0.00020799975214914443
キャッチコピー: 福岡の美味しいものが揃ってい  		 識別器の確率: 0.1317006 		 tf-idf値: 0.0


# 識別器の確率とtfidfを組み合わせた総合的なスコア付け

識別器の確率とtf-idfの値を組み合わせることで、キャッチコピーの総合的な完成度を表す。

組み合わせには、正規化した両方の値の調和平均を用いた。

In [17]:
import statistics

haiku_score_list = haiku_list

haiku_list_fin = []
tfidf_list = []
dis_list = []

for i in haiku_score_list:
    haiku_list_fin.append(i[0])
    dis_list.append(float(i[1]))
    tfidf_list.append(float(i[2]))

def min_max(l):
    l_min = min(l)
    l_max = max(l)
    return [(i - l_min) / (l_max - l_min) for i in l]

def standardization(l):
    l_mean = statistics.mean(l)
    l_stdev = statistics.stdev(l)
    return [(i - l_mean) / l_stdev for i in l]

# 標準化
# tfidf_list_std = standardization(tfidf_list)
# dis_list_std = standardization(dis_list)
# print(tfidf_list_std)

# 調和平均では値がマイナスだと困るため正規化を用いる
tfidf_list_std = min_max(tfidf_list)
dis_list_std = min_max(dis_list)
# print(tfidf_list_std)

[0.0, 0.09608402714198873, 0.5411499761412639, 0.006644488038804265, 0.0, 0.0, 0.5653664485832532, 0.023112427754006733, 0.00603039897947538, 0.0, 0.16628560682632432, 0.24299509129823596, 0.47980952285470063, 0.01631615555241318, 0.021805337342851772, 0.4130823300940636, 0.5653664485832532, 0.24299509129823596, 0.24299509129823596, 0.31837507854167824, 0.0, 0.01848309332344964, 0.00845391800720875, 0.05934792710971882, 0.07424850011890544, 0.08774822741325188, 0.006749863647173221, 0.5653664485832532, 0.06074877282455899, 0.0, 0.13612876006800123, 0.0008305610048505331, 0.24299509129823596, 0.06074877282455899, 0.24592819577593406, 0.0, 0.013499727294346443, 0.16132971695840276, 0.5653664485832532, 0.010422084546574258, 0.0, 0.4130823300940636, 0.03541616097435091, 0.022545325763559512, 0.07424850011890544, 0.0, 0.07537998724344225, 0.01631615555241318, 0.006644488038804265, 0.01206079795895076, 0.4130823300940636, 0.41852104861153466, 0.06749863647173222, 0.4130823300940636, 0.008453

調和平均を求める

In [18]:
sum_list = [0]*len(tfidf_list_std)

# 調和平均を計算
# 識別器の確率かtfidfのどちらかが0の場合は、調和平均を0にする
for i in range(len(tfidf_list_std)):
    if tfidf_list_std[i] == 0 or dis_list_std[i] == 0:
        sum_list[i] = 0
    else:
        sum_list[i] = 2/(1/tfidf_list_std[i] + 1/dis_list_std[i])

# print(sum_list)

[0, 0.17303496477142397, 0.6241658019042455, 0.013163449335735355, 0, 0, 0.31481923477463325, 0.041540615332564534, 0.011586470500946362, 0, 0.14164687373901252, 0.14517734959389478, 0.13768295165397126, 0.02638947508359181, 0.03299454037829724, 0.11163222012888578, 0.11529563937951873, 0.094058178641278, 0.093290145989042, 0.09396530397602362, 0, 0.027085199411670906, 0.014441128137855728, 0.05153062745260041, 0.05556343671528252, 0.05895731082480468, 0.011701556716858198, 0.07809120061471288, 0.04884959752380007, 0, 0.062286327775097586, 0.001627173220920884, 0.06807156734576096, 0.04709312550453226, 0.06499663057347074, 0, 0.019655806689244274, 0.058103165699126116, 0.06387963289134495, 0.015916012793348134, 0, 0.05954775798011057, 0.03364210882154876, 0.026462437240233932, 0.04413056428675509, 0, 0.043981248873862844, 0.021369204648615945, 0.010940571205093888, 0.01731166102972886, 0.05530116603988069, 0.055147698790540475, 0.04068706408910845, 0.052544219230473595, 0.0129688356792

生成例とそれに対するスコアを表示

識別器の確率とtfidfの値は、正規化されているため0~1で表示される

In [22]:
score_list = []
for i in range(len(sum_list)):
    temp = [haiku_list_fin[i],dis_list_std[i],tfidf_list_std[i],sum_list[i]]
    score_list.append(temp)

for n in score_list[:10]:
    print("生成例:", n[0], "\t", "識別器:", n[1], "\t", "tf-idf:", n[2], "\t", "調和平均:", n[3])

# スコアを保存
import pickle

with open('data/generated/hakataeki/score/score.pickle', 'wb') as f:
    pickle.dump(score_list, f)

生成例: アクセスが良い駅だなと思いまし 	 識別器: 1.0 	 tf-idf: 0.0 	 調和平均: 0
生成例: 駅前の道路の工事渋滞で 	 識別器: 0.8689603548189178 	 tf-idf: 0.09608402714198873 	 調和平均: 0.17303496477142397
生成例: お土産は１Fみやげもん市場 	 識別器: 0.7372672579194987 	 tf-idf: 0.5411499761412639 	 調和平均: 0.6241658019042455
生成例: イベントもあるし１日遊べます 	 識別器: 0.6967788724180182 	 tf-idf: 0.006644488038804265 	 調和平均: 0.013163449335735355
生成例: 雰囲気が良い駅だなと思いまし 	 識別器: 0.5813069520327749 	 tf-idf: 0.0 	 調和平均: 0
生成例: シーズンになるとたくさんありました 	 識別器: 0.2649736620400159 	 tf-idf: 0.0 	 調和平均: 0
生成例: 地下鉄も新幹線でつながって 	 識別器: 0.21814591532694091 	 tf-idf: 0.5653664485832532 	 調和平均: 0.31481923477463325
生成例: 全体が数年前にリニューアル 	 識別器: 0.20496482562166374 	 tf-idf: 0.023112427754006733 	 調和平均: 0.041540615332564534
生成例: 福岡の窓口である博多駅 	 識別器: 0.14730549266237247 	 tf-idf: 0.00603039897947538 	 調和平均: 0.011586470500946362
生成例: 福岡の美味しいものが揃ってい 	 識別器: 0.13413396367079627 	 tf-idf: 0.0 	 調和平均: 0


# 各スコアでの上位のキャッチコピー

各スコア付けでの上位１０件のキャッチコピーを表示

In [29]:
import pickle
with open('data/generated/hakataeki/score/score.pickle', 'rb') as f:
    score_list = pickle.load(f)

In [30]:
# 識別器の確率によるスコアでの上位のキャッチコピー

score_list_10 = sorted(score_list, key=lambda x: x[1])
score_list_10.reverse()
# score_list_10.pop(4)
score_list_10 = score_list_10[:10]

print("識別器の確率によるスコアでの上位のキャッチコピー")
print("")
for n in score_list_10:
    print("生成例:", n[0], "\t", "識別器:", n[1])

識別器の確率によるスコアでの上位のキャッチコピー

生成例: アクセスが良い駅だなと思いまし 	 識別器: 1.0
生成例: 駅前の道路の工事渋滞で 	 識別器: 0.8689603548189178
生成例: お土産は１Fみやげもん市場 	 識別器: 0.7372672579194987
生成例: イベントもあるし１日遊べます 	 識別器: 0.6967788724180182
生成例: 雰囲気が良い駅だなと思いまし 	 識別器: 0.5813069520327749
生成例: シーズンになるとたくさんありました 	 識別器: 0.2649736620400159
生成例: 地下鉄も新幹線でつながって 	 識別器: 0.21814591532694091
生成例: 全体が数年前にリニューアル 	 識別器: 0.20496482562166374
生成例: 福岡の窓口である博多駅 	 識別器: 0.14730549266237247
生成例: 福岡の美味しいものが揃ってい 	 識別器: 0.13413396367079627


In [31]:
# tfidfによるスコアでの上位のキャッチコピー

score_list_10 = sorted(score_list, key=lambda x: x[2])
score_list_10.reverse()
# score_list_10.pop(4)
score_list_10 = score_list_10[:10]

print("tfidfによるスコアでの上位のキャッチコピー")
print("")
for n in score_list_10:
    print("生成例:", n[0], "\t", "tfidf:", n[2])

tfidfによるスコアでの上位のキャッチコピー

生成例: お土産に直結なのでお土産に 	 tfidf: 1.0
生成例: 阪急に直結なのでお土産に 	 tfidf: 0.9336362139829014
生成例: 地下鉄も新幹線で行きました 	 tfidf: 0.5653664485832532
生成例: 地下鉄も新幹線で開催て 	 tfidf: 0.5653664485832532
生成例: 地下鉄も新幹線で取りました 	 tfidf: 0.5653664485832532
生成例: 地下鉄も新幹線でつながって 	 tfidf: 0.5653664485832532
生成例: お土産や１Fみやげもん市場 	 tfidf: 0.5411499761412639
生成例: お土産は１Fみやげもん市場 	 tfidf: 0.5411499761412639
生成例: 境内に直結なのでお土産に 	 tfidf: 0.528644395152508
生成例: 散策に直結なのでお土産に 	 tfidf: 0.528644395152508


In [32]:
# 調和平均によるスコアでの上位のキャッチコピー

score_list_10 = sorted(score_list, key=lambda x: x[3])
score_list_10.reverse()
# score_list_10.pop(4)
score_list_10 = score_list_10[:10]

print("調和平均によるスコアでの上位のキャッチコピー")
print("")
for n in score_list_10:
    print("生成例:", n[0], "\t", "調和平均:", n[3])

調和平均によるスコアでの上位のキャッチコピー

生成例: お土産は１Fみやげもん市場 	 調和平均: 0.6241658019042455
生成例: 地下鉄も新幹線でつながって 	 調和平均: 0.31481923477463325
生成例: 駅前の道路の工事渋滞で 	 調和平均: 0.17303496477142397
生成例: 福岡の玄関口につき人が 	 調和平均: 0.14517734959389478
生成例: 地下鉄も外国人の思入れ 	 調和平均: 0.14164687373901252
生成例: お土産を外国からのお客様 	 調和平均: 0.13768295165397126
生成例: 地下鉄も新幹線で取りました 	 調和平均: 0.11529563937951873
生成例: イベントに新幹線で開催さ 	 調和平均: 0.11163222012888578
生成例: 九州の玄関口でつき人が 	 調和平均: 0.094058178641278
生成例: 九州の玄関口でJR 	 調和平均: 0.09396530397602362
